In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import numpy as np
import pandas as pd
import time

In [34]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
class utilities:
    def wait(self, seconds):

        s = np.random.normal(0, 1)
        time.sleep(seconds*abs(s))
    
    def get_element_by_path(self, driver, xpath):
        try:
            return driver.find_element_by_xpath(xpath)
        except Exception as e:
            print(e)
            return None

In [81]:
class IndeedItemParser:
    def __init__(self):
        self.driverPath = "C:\\Users\\Junior\\Documents\\selenium\\driver\\chromedriver.exe"
        
        
    def _get_title(self, driver):
        try:
            title = driver.find_element_by_xpath("//*[@class='jobsearch-DesktopStickyContainer']//h3")
        except Exception as e:
            print(e)
        return title.text
    
    def _get_name(self, driver):
        
        try:    
            name = driver.find_element_by_xpath("//*[contains(@class,'jobsearch-InlineCompanyRating')]//div[1]")
            return name.text
        except Exception as e:
            print(e)
        
    
    def _get_address(self,driver):
         try:   
            address = driver.find_element_by_xpath("//*[contains(@class,'jobsearch-InlineCompanyRating')]//div[3]")
            if address.text == "-":
                address = driver.find_element_by_xpath("//*[contains(@class,'jobsearch-InlineCompanyRating')]//div[4]")
            return address.text
         except Exception as e:
            address = driver.find_element_by_xpath("//span[@class='jobsearch-JobMetadataHeader-iconLabel'][1]")
            return address.text
    
    def _get_description(self,driver):
        try:
            #jobDescriptionText
            e_description = driver.find_element_by_id("jobDescriptionText")
            return e_description.get_attribute('innerHTML')
        except Exception as e:
            print(e)
    
    def _get_date(self,driver,url,name):
       # print(url)
        try:
            date_str = driver.find_element_by_xpath("//*[@class='jobsearch-JobMetadataFooter']")
            date_str_full = date_str.text
            date_tbl = date_str_full.split(" ")
            count_str = date_tbl[4]
            label = date_tbl[5]
            
            if name in date_str_full:
                date_str_full = date_str_full.replace(name, "")
                date_tbl = date_str_full.split(" ")
                count_str = date_tbl[5]
                label = date_tbl[6]
                
            if count_str == "a" :
                count_str = date_tbl[5]
                label = date_tbl[6]
                
            #print("date_str", date_tbl)
            date = datetime.now()
            
            if count_str == "30+":
                return date - timedelta(days=30)
            
            count = int(count_str)
            if "jour" in label:
                date = date - timedelta(days=count)
            elif "heur" in label:
                date = date - timedelta(hours=count)
            return date;
        except Exception as e:
            print(e)
    
    
    def parse(self,url):
        driver = webdriver.Chrome(self.driverPath)
        driver.get(url)
        driver.maximize_window()
        
        title = self._get_title(driver)
        name = self._get_name(driver)
        address = self._get_address(driver)
        date = self._get_date(driver, url,name)
        description = self._get_description(driver)
        
        driver.close()
        
        return title, name, address, date, description

In [82]:
import os.path
from os import path
import json

class IndeedParserStateObject(object):
    def __init__(self, data):
        self.__dict__ = json.loads(data)
        
class IndeedParserStateHandler:
    
    def __init__(self):
        self.filename = "indeed.parser.state.json"
        self.object = self._load_file()
        print(type(self.object))
        #self.object = IndeedParserStateObject(self.data)
    
    def save_state(self, job, location, page_request="", page_item_index=-1):
        self.object["job"] = job
        self.object["location"] = location
        self.object["page_request"] = page_request
        self.object["page_item_index"]  = page_item_index
        #self.data  = json.dumps(self.object)
        
        #data = json.dumps(self.object)
        self._save_file(self.object)
        #self.object = IndeedParserStateObject(self.data)
        
    def is_current_job(self, job):
        if (self.object["job"] == "") | (self.object["job"] == job):
            return True
        return False
    
    def is_current_location(self, location):
        if (self.object["location"] == "") | (self.object["location"] == location):
            return True
        return False
    
    def is_current_page_request(self, page_request):
        if (self.object["page_request"] == "") | (self.object["page_request"] == page_request):
            return True
        return False
    
    def _save_file(self, data):
         with open(self.filename, 'w') as outfile:
                json.dump(data, outfile)
                
    def _load_file(self):
        
        if path.exists(self.filename) == True:
            with open(self.filename) as json_file:
                data = json.load(json_file)
        else:
            data = {
                     "job": "",
                     "location":"",
                     "page_request":"",
                     "page_item_index":-1,
                    }
            with open(self.filename, 'w') as outfile:
                json.dump(data, outfile)
        
        return data


In [83]:
from datetime import datetime, timedelta
class IndeedPaser:
    def __init__(self):
        self.website = "https://www.indeed.fr"
        self.driverPath = "C:\\Users\\Junior\\Documents\\selenium\\driver\\chromedriver.exe"
        #self.dataset = pd.DataFrame(columns=['URL', 'Titre','Nom entreprise','Adresse','Date de publication', 'description'])
        self.dataset = pd.read_csv("indeed.csv")
        self.jobs = ["développeur", "data scientist", "data analyst", "business intelligence"]
        self.locations = ["Paris", "Lyon", "Toulouse", "Nantes", "Bordeaux"]
        self.utilities =  utilities()
        self.indeed_item_parser = IndeedItemParser()
        self.parser_state = IndeedParserStateHandler()
    
    def _get_pages_counts(self,driver):
        #searchCountPages
        searchCountPages_elt = driver.find_element_by_id("searchCountPages")
        searchCountPages = searchCountPages_elt.text.split()
        if len(searchCountPages) == 6:
            searchCountPages = int("{0}{1}".format(searchCountPages[3],searchCountPages[4])) 
        else :
            searchCountPages = searchCountPages[3]  
                    
        return (int)(searchCountPages / 18)
    
    def parse(self):
        browser = webdriver.Chrome(self.driverPath)
        browser.get(self.website)
        browser.maximize_window()
        
        for job in self.jobs:
            
            if self.parser_state.is_current_job(job) == False:
                continue
                    
            for location in self.locations:
                
                if self.parser_state.is_current_location(location) == False:
                    continue
                self.parser_state.save_state(job, location)
                
                query = "https://www.indeed.fr/jobs?q={0}&l={1}".format(job, location)
                browser.get(query)
                    
                pages_count = self._get_pages_counts(browser)
                
                for page_index in range(1, pages_count):
                    full_query = "{0}&start={1}".format(query,page_index)
                    
                    browser.get(full_query)
                    
                    items = browser.find_elements_by_xpath("//*[contains(@class,'clickcard')]")
                    dataset_len = len(self.dataset)
                    
                    for index_i,item in enumerate(items): 
                        title = item.find_element_by_xpath(".//*[contains(@class,'jobtitle')]")
                        item_link = title.get_attribute("href")
                        
                        if (len(self.dataset) == 0) | (True not in self.dataset.isin([item_link])):
                            print("existe pas")
                            title, name, address, date, description = self.indeed_item_parser.parse(item_link)
                            self.dataset.loc[dataset_len + index_i] = [item_link, title, name, address, date,description]
                            print("len(dataset)", len(self.dataset))
                        else:
                            print("existe déjà")
                        
                        #break
                    #break
                    self.dataset.to_csv("indeed.csv", index=False)
                    #https://www.indeed.fr/jobs?q=developpeur&l=paris&start=10
                    
                print(searchCountPages)
                #break
            #break         

In [10]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install django

Note: you may need to restart the kernel to use updated packages.


In [84]:
class CsvToMongodbMigratorTool:
    
    def __init__(self):
        self.dataset = pd.read_csv("indeed.csv")
        self.mongodbdao = IndeedMongodbDao()
        
    def migrate(self):
        records = json.loads(self.dataset.T.to_json()).values()
        self.mongodbdao.insert_data_bulk(records)
        

In [56]:
data_set = pd.read_csv("indeed.csv")
record = data_set.iloc[1]

indeeddao = IndeedMongodbDao()
indeeddao.insert_data(record["URL"],record["Titre"], record["Nom entreprise"],record["Adresse"], record["Date de publication"], record["description"])


DuplicateKeyError : url https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYlbfkN0BPA0_wx9cDprZ7BJ5N77NUbc2ajC2I_x_zlGmtNnAwCfgmTUExHA28Yma9NzXrLsTN4stEHdEuocdTfP8hBmaakm9ZUCKPYcZAg0YExzTak9_N3-S77YYoZtvUW0y4EHqK1Kwq_IHMV2MKxNx2-t-vFGzsYTvzYwt6y5UIJJEAdl4smulknsMbk53xIFDBmKKoWawBCgxb3OqTnRUgEPYxwWpsBhGbYAd2kix18RagMVF9xcNKc2JPHTu-rcNMCwTivg5h5Qteg0-iNNja976hKLm_JYOwEtg42Q7k7Sc9Td_xwXeqqfQRrm5jYgyGxlJfSzZt9XRBrJh0RNxovDLGobKvEop9UvXNb8oy_RApP6L6KdTXVtIJxzihBGawwmxMrj4P-icqG_jtX3bx-nxoo5fP-mC3_hZCXM90af14FGLkhZ0nO34tbymzMVhw4-qTe6Sn7L0Ay9u1hi7kjox4-9L9WKyoUJ1ONj517p1zHc2TXeSoGyz2vO5NiOZCRlk=&p=1&fvj=1&vjs=3 aready exist


In [76]:
y = 'https://www.indeed.fr/company/LEC-Conseil/jobs/D%C3%A9veloppeur-Fullstack-F-H-ad4f8380f4e66e4a?fccid=3d3e7ac112fae637&vjs=3'
if True not in data_set["URL"].isin([y]):
    print("yes")

In [67]:
data_set["URL"].iloc[10]

'https://www.indeed.fr/company/LEC-Conseil/jobs/D%C3%A9veloppeur-Fullstack-F-H-ad4f8380f4e66e4a?fccid=3d3e7ac112fae637&vjs=3'

In [78]:
migratorTool = CsvToMongodbMigratorTool()
migratorTool.migrate()

BulkWriteError: batch op errors occurred

In [85]:
from pymongo import MongoClient 
from pymongo import errors 
from django.core.validators import URLValidator
from django.core.exceptions import ValidationError
class IndeedMongodbDao:
    def __init__(self):
        self.conn = MongoClient() 
        self.db = self.conn.Indeed
        self.collection = self.db.data
        self._create_index()
    
    def _create_index(self):
        index_name = 'url'
        if index_name not in self.collection.index_information():
            self.collection.create_index(index_name, unique=True)
    
    def _valid_url_format(self,url):
        val = URLValidator()
        try:
            val(url)
        except ValidationError as e:
            raise Exception('bad format for url {}'.format(ur))
    
    def insert_data_bulk(self,data):
        self.collection.insert_many(data)
    
    def insert_data(self, url, title, name, address, publication_date, description):
        
        try:
            if url == "":
                raise Exception('url cannot be empty {}'.format(ur))

            self._valid_url_format(url)

            if title == "":
                raise Exception('title cannot be empty {}'.format(title))

            if name == "":
                raise Exception('the name of company cannot be be empty {}'.format(title))

            if description == "":
                raise Exception('description of company cannot be be empty {}'.format(title))


            line_to_insert = {
                                "URL": url,
                                "title":title,
                                "name":name,
                                "adresse":address,
                                "publication_date":publication_date,
                                "description":description
                             }

            # Insert Data 
            result = self.collection.insert_one(line_to_insert) 
        except errors.DuplicateKeyError as e:
            print('DuplicateKeyError : url {0} aready exist'.format(url))
    
    def get_all_data(self):
        data = self.collection.find({})
        return data
    
    

In [52]:
item_parser = IndeedItemParser()
result = item_parser.parse("https://www.indeed.fr/voir-emploi?jk=9dc37ec01c9984dd&from=serp&vjs=3")
print(result)

('Développeur front-end', 'Be Free Talent', 'Paris (75)', datetime.datetime(2019, 9, 26, 22, 19, 25, 92621), '<div><p>Emploi</p>\n<p><b>\nFormation :</b> un bac + 2 dans une école d’informatique est largement suffisant pour accéder à ce métier ou une reconversion professionnelle <i>via</i> un centre de formation informatique <b>(formation intensive de 3 à 6 mois)</b> permet d’accéder au métier de développeur front-end ou consultant<i> front-end</i>. Et pour les plus malins, des cours d’<i>openclassrooms</i> sont largement suffisants.</p>\n<p><b>\nActivité :</b> le développeur front-end travaille sur des projets de refonte et/ou création de site web.<br>\nTrès souvent, il dispose des compétences en design qui le permet de réaliser des maquettes du futur site internet.<br>\nIl doit disposer de solides compétences en <b>HTML &amp; CSS et jQuery.</b></p>\n<p><b>\nÉvolution :</b> après quelques expériences professionnelles sur du front-end, le développeur front-end peut monter en compétence

In [ ]:
test = IndeedPaser()
test.parse()

<class 'dict'>
existe pas
len(dataset) 1
existe pas
len(dataset) 2
existe pas
len(dataset) 3
existe pas
len(dataset) 4
existe pas
len(dataset) 5
existe pas
len(dataset) 6
existe pas


In [7]:
test.dataset

,URL,Titre,Nom entreprise,Adresse,Date de publication,description
0,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Développeur JAVA/J2EE,Activ Browser Technology,Boulogne-Billancourt (92),2019-10-01 15:41:50.559427,"<p>Activ Browser, la créativité d’une Web Agen..."
1,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Développeur Software 3D / Unity 3D / RM & RA /...,Theoris,75010 Paris 10e,2019-09-02 15:42:08.057680,<p>Vous êtes passionné de <b>Développement Sof...
2,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Développeur Ionic / Firebase,NEXTT,92200 Neuilly-sur-Seine,2019-09-18 15:42:22.787673,<p>Dans le cadre du développement d'une applic...
3,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Développeur Java H/F,Ratp Smart Systems,Paris (75),2019-09-09 15:42:39.629319,<p><b>1er jet : </b></p><p>Pour accélérer sa t...
4,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Développeur Android (H/F),Universign,Paris (75),2019-09-02 15:42:50.657003,<div><div><b>Universign</b> est le pure player...
5,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Développeur Informatique H/F,VALENTE SECURYSTAR,94320 Thiais,2019-10-01 16:43:07.533091,"<p>VALENTE SECURYSTAR, fabricant de portes bli..."
6,https://www.indeed.fr/rc/clk?jk=846f7f5da2e3fa...,DÉVELOPPEUR(-EUSE) PRODUIT PAP (F/H) - CDD,Louis Vuitton,Paris (75),2019-09-19 15:43:18.703088,"<div><div><h2 class=""jobSectionHeader""><b>Post..."
7,https://www.indeed.fr/rc/clk?jk=b5268c024d6547...,Assistant développeur Front (H/F),OUI.sncf,Paris (75),2019-09-27 15:43:31.457057,<div><p>SIDH (Service d’Itinéraires et de Donn...
8,https://www.indeed.fr/company/CREATIVE-INGENIE...,Développeur PHP H/F,CREATIVE INGENIERIE,Paris 6e (75),2019-09-24 15:43:42.364257,<p><b>LES MISSIONS QUI TE SERONT CONFIEES : </...
9,https://www.indeed.fr/rc/clk?jk=a88bee6c673082...,Développeur iOS (H/F),Meetic Group,Paris (75),2019-10-02 05:43:55.761557,<div>Job Description<br>\n<br>\n<p>Derrière la...
